<a href="https://colab.research.google.com/github/Farhannr28/K01-IF2220-13522037-13522039/blob/main/Tugas_Besar_Probstat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset Health
## Farhan Nafis Rayhan - 13522037
## Edbert Eddyson Gunawan - 13522039


## Setup Dataset

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

In [ ]:
main_df = pd.read_csv('health.csv')
main_df.head()

,Unnamed: 0,Age,Income,Gender,Education,Stress_Level,Exercise_Hours_Per_Week,Cholesterol_Level,Weight,Height,Blood_Pressure,Health_Status
0,0,50,3.093457e+06,Male,Bachelor's Degree,4.967887,16.632494,200.419036,70.917032,150.401537,125.568820,Excellent
1,1,44,5.545445e+06,Male,Bachelor's Degree,5.833649,13.255988,201.010046,70.669576,149.285349,125.603505,Fair
2,2,25,4.401808e+06,Male,Master's Degree,5.587946,11.144370,200.105258,67.464506,150.020695,127.093163,Excellent
3,3,41,4.606865e+06,Female,High School,6.271119,9.140268,199.648701,67.566598,150.360963,117.388100,Poor
4,4,24,6.956049e+06,Male,Bachelor's Degree,5.895226,9.775610,199.330152,79.601530,149.730525,114.163706,Fair


In [ ]:
main_df.describe()

,Unnamed: 0,Age,Income,Stress_Level,Exercise_Hours_Per_Week,Cholesterol_Level,Weight,Height,Blood_Pressure
count,2000.000000,2000.000000,2.000000e+03,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,999.500000,39.417500,4.889928e+06,5.356912,9.951842,200.223191,70.234359,150.730565,119.682437
std,577.494589,11.560753,2.010795e+06,0.917069,4.909573,0.972594,10.174343,0.680443,11.201412
min,0.000000,20.000000,-1.088987e+06,2.528057,-4.499310,196.914948,32.581758,149.124594,0.000000
25%,499.750000,30.000000,3.580714e+06,4.737650,6.654951,199.581975,63.654894,150.240019,113.048602
50%,999.500000,39.000000,4.898901e+06,5.351795,9.927028,200.221522,70.392125,150.663619,119.937396
75%,1499.250000,49.250000,6.271595e+06,6.003014,13.232218,200.873647,76.894512,151.140489,126.608781
max,1999.000000,59.000000,1.222749e+07,8.581327,26.816358,203.162335,109.609562,153.870469,200.950000


In [ ]:
main_df.dtypes

Unnamed: 0                   int64
Age                          int64
Income                     float64
Gender                      object
Education                   object
Stress_Level               float64
Exercise_Hours_Per_Week    float64
Cholesterol_Level          float64
Weight                     float64
Height                     float64
Blood_Pressure             float64
Health_Status               object
dtype: object

## **1. Deskripsi statistika**

Menulis deskripsi statistika (Descriptive Statistics) dari semua kolom pada data. Data yang bersifat numerik dapat diberikan nilai mean, median, modus, standar deviasi, variansi, range, nilai minimum, maksimum, kuartil, IQR, skewness dan kurtosis. Data dalam bentuk string dapat dicari unique values, dan proporsi nya.


### Fungsi Buatan

In [ ]:
def getMean(column: pd.core.series.Series):
  if(column.dtypes != 'O'):
    return sum(column) / len(column)

def getMedian(column: pd.core.series.Series):
  if(column.dtypes != 'O'):
    column = column.sort_values()
    mid = column.size // 2

    if column.size % 2 == 0:
      return (column.iloc[mid-1] + column.iloc[mid]) / 2

    return column[mid]

def getModus(column: pd.core.series.Series):
  dictionary_element = {}
  for x in column:
    if x in dictionary_element:
      dictionary_element[x] += 1
    else:
      dictionary_element.setdefault(x, 1)
  # print(column.mode())
  return max(dictionary_element, key=lambda k: dictionary_element[k])

def getStd(column: pd.core.series.Series):
  std_dev = getVariance(column) ** 0.5
  return std_dev

def getVariance(column: pd.core.series.Series):
  mean_val = getMean(column)
  squared_diff = [(x - mean_val) ** 2 for x in column]
  variance = sum(squared_diff) / len(column)
  return variance

def getRange(column: pd.core.series.Series):
  return getMax(column) - getMin(column)

def getMax(column: pd.core.series.Series):
  return max(column)

def getMin(column: pd.core.series.Series):
  return min(column)

def getKuartil(column: pd.core.series.Series):
  column = column.sort_values()

  q2 = getMedian(column)
  lower_half = [x for x in column if x < q2]
  upper_half = [x for x in column if x > q2]
  lower_half = pd.Series(lower_half)
  upper_half = pd.Series(upper_half)
  q1 = getMedian(lower_half)
  q3 = getMedian(upper_half)
  return [q1, q2, q3]

def getInterKuartil(column: pd.core.series.Series):
  quartile = getKuartil(column)
  return quartile[2] - quartile[0]

def getSkewness(column: pd.core.series.Series):
  return sum((x - getMean(column)) ** 3 for x in column) / (len(column) * getStd(column) ** 3)

def kurtosis(column: pd.core.series.Series):
  return sum((x - getMean(column)) ** 4 for x in column) / (len(column) * getStd(column) ** 4) - 3

def getUnique(column: pd.core.series.Series):
  unique_set = set()
  for x in column:
    unique_set.add(x)
  return unique_set

def getProportion(column: pd.core.series.Series):
  dictionary_element = {}
  for x in column:
    if x in dictionary_element:
      dictionary_element[x] += 1
    else:
      dictionary_element.setdefault(x, 1)

  sum_val = sum(dictionary_element.values())
  for x in dictionary_element:
    dictionary_element[x] /= sum_val

  return dictionary_element

def showInfo(column: pd.core.series.Series):
  if(column.dtypes != 'O'):
    getMean(column)
    getMedian(column)
  else:
    getUnique()
    getProportion()

### A) Deskripsi Statistika Income

In [ ]:
main_df['Age']

0       3.093457e+06
1       5.545445e+06
2       4.401808e+06
3       4.606865e+06
4       6.956049e+06
            ...     
1995    2.072546e+06
1996    7.456909e+06
1997    5.560391e+06
1998    3.436759e+06
1999    3.343102e+06
Name: Income, Length: 2000, dtype: float64

In [ ]:
x = getMean(main_df['Age'])
print("Rata-rata: " + str(x))
x = getMedian(main_df['Age'])
print("Median: " + str(x))
x = getModus(main_df['Age'])
print("Modus: " + str(x))
x = getStd(main_df['Age'])
print("Standar Deviasi: " + str(x))
x = getVariance(main_df['Age'])
print("Variansi: " + str(x))
x = getRange(main_df['Age'])
print("Range: " + str(x))
x = getMin(main_df['Age'])
print("Nilai Minimum: " + str(x))
x = getMax(main_df['Age'])
print("Nilai Maksimum: " + str(x))
x = getKuartil(main_df['Age'])
print("Kuartil: " + str(x))
x = getInterKuartil(main_df['Age'])
print("Inter Kuartil: " + str(x))
x = getSkewness(main_df['Age'])
print("Skewness: " + str(x))
x = kurtosis(main_df['Age'])
print("Kurtosis: " + str(x))

Rata-rata: 4889928.319474324
Median: 4898900.675226251
Modus: 3093456.839129376
Standar Deviasi: 2010292.2629119065
Variansi: 4041274982323.4736
Range: 13316478.756404964
Nilai Minimum: -1088986.5264049638
Nilai Maksimum: 12227492.23
Kuartil: [3579592.598647509, 4898900.675226251, 6272831.370186756]
Inter Kuartil: 2693238.7715392476
Skewness: -0.006346172850509155
Kurtosis: -0.03448157628069426


### B) Deskripsi Statistika Income

In [ ]:
main_df['Income']

0       3.093457e+06
1       5.545445e+06
2       4.401808e+06
3       4.606865e+06
4       6.956049e+06
            ...     
1995    2.072546e+06
1996    7.456909e+06
1997    5.560391e+06
1998    3.436759e+06
1999    3.343102e+06
Name: Income, Length: 2000, dtype: float64

In [ ]:
x = getMean(main_df['Income'])
print("Rata-rata: " + str(x))
x = getMedian(main_df['Income'])
print("Median: " + str(x))
x = getModus(main_df['Income'])
print("Modus: " + str(x))
x = getStd(main_df['Income'])
print("Standar Deviasi: " + str(x))
x = getVariance(main_df['Income'])
print("Variansi: " + str(x))
x = getRange(main_df['Income'])
print("Range: " + str(x))
x = getMin(main_df['Income'])
print("Nilai Minimum: " + str(x))
x = getMax(main_df['Income'])
print("Nilai Maksimum: " + str(x))
x = getKuartil(main_df['Income'])
print("Kuartil: " + str(x))
x = getInterKuartil(main_df['Income'])
print("Inter Kuartil: " + str(x))
x = getSkewness(main_df['Income'])
print("Skewness: " + str(x))
x = kurtosis(main_df['Income'])
print("Kurtosis: " + str(x))

Rata-rata: 4889928.319474324
Median: 4898900.675226251
Modus: 3093456.839129376
Standar Deviasi: 2010292.2629119065
Variansi: 4041274982323.4736
Range: 13316478.756404964
Nilai Minimum: -1088986.5264049638
Nilai Maksimum: 12227492.23
Kuartil: [3579592.598647509, 4898900.675226251, 6272831.370186756]
Inter Kuartil: 2693238.7715392476
Skewness: -0.006346172850509155
Kurtosis: -0.03448157628069426


### C) Deskripsi Statistika Gender

In [ ]:
main_df['Gender']

0         Male
1         Male
2         Male
3       Female
4         Male
         ...  
1995      Male
1996    Female
1997      Male
1998      Male
1999      Male
Name: Gender, Length: 2000, dtype: object

In [ ]:
x = getUnique(main_df['Gender'])
if len(x)==0 :
  print("Tidak ada value unik")
else:
  print("Value Unik: " + str(x))

x = getProportion(main_df['Gender'])
print("Proporsi: " + str(x))

Value Unik: {'Male', 'Female'}
Proporsi: {'Male': 0.4945, 'Female': 0.5055}


### D) Deskripsi Statistika Education

In [ ]:
main_df['Education']

0         Male
1         Male
2         Male
3       Female
4         Male
         ...  
1995      Male
1996    Female
1997      Male
1998      Male
1999      Male
Name: Gender, Length: 2000, dtype: object

In [ ]:
x = getUnique(main_df['Education'])
if len(x)==0 :
  print("Tidak ada value unik")
else:
  print("Value Unik: " + str(x))

x = getProportion(main_df['Education'])
print("Proporsi: " + str(x))

Value Unik: {'Male', 'Female'}
Proporsi: {'Male': 0.4945, 'Female': 0.5055}


### E) Deskripsi Statistika Stress Level

In [ ]:
main_df['Stress_Level']

0       3.093457e+06
1       5.545445e+06
2       4.401808e+06
3       4.606865e+06
4       6.956049e+06
            ...     
1995    2.072546e+06
1996    7.456909e+06
1997    5.560391e+06
1998    3.436759e+06
1999    3.343102e+06
Name: Income, Length: 2000, dtype: float64

In [ ]:
x = getMean(main_df['Stress_Level'])
print("Rata-rata: " + str(x))
x = getMedian(main_df['Stress_Level'])
print("Median: " + str(x))
x = getModus(main_df['Stress_Level'])
print("Modus: " + str(x))
x = getStd(main_df['Stress_Level'])
print("Standar Deviasi: " + str(x))
x = getVariance(main_df['Stress_Level'])
print("Variansi: " + str(x))
x = getRange(main_df['Stress_Level'])
print("Range: " + str(x))
x = getMin(main_df['Stress_Level'])
print("Nilai Minimum: " + str(x))
x = getMax(main_df['Stress_Level'])
print("Nilai Maksimum: " + str(x))
x = getKuartil(main_df['Stress_Level'])
print("Kuartil: " + str(x))
x = getInterKuartil(main_df['Stress_Level'])
print("Inter Kuartil: " + str(x))
x = getSkewness(main_df['Stress_Level'])
print("Skewness: " + str(x))
x = kurtosis(main_df['Stress_Level'])
print("Kurtosis: " + str(x))

Rata-rata: 4889928.319474324
Median: 4898900.675226251
Modus: 3093456.839129376
Standar Deviasi: 2010292.2629119065
Variansi: 4041274982323.4736
Range: 13316478.756404964
Nilai Minimum: -1088986.5264049638
Nilai Maksimum: 12227492.23
Kuartil: [3579592.598647509, 4898900.675226251, 6272831.370186756]
Inter Kuartil: 2693238.7715392476
Skewness: -0.006346172850509155
Kurtosis: -0.03448157628069426


### F) Deskripsi Statistika Excercise Hours Per Week

In [ ]:
main_df['Exercise_Hours_Per_Week']

0       3.093457e+06
1       5.545445e+06
2       4.401808e+06
3       4.606865e+06
4       6.956049e+06
            ...     
1995    2.072546e+06
1996    7.456909e+06
1997    5.560391e+06
1998    3.436759e+06
1999    3.343102e+06
Name: Income, Length: 2000, dtype: float64

In [ ]:
x = getMean(main_df['Exercise_Hours_Per_Week'])
print("Rata-rata: " + str(x))
x = getMedian(main_df['Exercise_Hours_Per_Week'])
print("Median: " + str(x))
x = getModus(main_df['Exercise_Hours_Per_Week'])
print("Modus: " + str(x))
x = getStd(main_df['Exercise_Hours_Per_Week'])
print("Standar Deviasi: " + str(x))
x = getVariance(main_df['Exercise_Hours_Per_Week'])
print("Variansi: " + str(x))
x = getRange(main_df['Exercise_Hours_Per_Week'])
print("Range: " + str(x))
x = getMin(main_df['Exercise_Hours_Per_Week'])
print("Nilai Minimum: " + str(x))
x = getMax(main_df['Exercise_Hours_Per_Week'])
print("Nilai Maksimum: " + str(x))
x = getKuartil(main_df['Exercise_Hours_Per_Week'])
print("Kuartil: " + str(x))
x = getInterKuartil(main_df['Exercise_Hours_Per_Week'])
print("Inter Kuartil: " + str(x))
x = getSkewness(main_df['Exercise_Hours_Per_Week'])
print("Skewness: " + str(x))
x = kurtosis(main_df['Exercise_Hours_Per_Week'])
print("Kurtosis: " + str(x))

Rata-rata: 4889928.319474324
Median: 4898900.675226251
Modus: 3093456.839129376
Standar Deviasi: 2010292.2629119065
Variansi: 4041274982323.4736
Range: 13316478.756404964
Nilai Minimum: -1088986.5264049638
Nilai Maksimum: 12227492.23
Kuartil: [3579592.598647509, 4898900.675226251, 6272831.370186756]
Inter Kuartil: 2693238.7715392476
Skewness: -0.006346172850509155
Kurtosis: -0.03448157628069426


### G) Deskripsi Statistika Cholesterol Level

In [ ]:
main_df['Cholesterol_Level']

0       3.093457e+06
1       5.545445e+06
2       4.401808e+06
3       4.606865e+06
4       6.956049e+06
            ...     
1995    2.072546e+06
1996    7.456909e+06
1997    5.560391e+06
1998    3.436759e+06
1999    3.343102e+06
Name: Income, Length: 2000, dtype: float64

In [ ]:
x = getMean(main_df['Cholesterol_Level'])
print("Rata-rata: " + str(x))
x = getMedian(main_df['Cholesterol_Level'])
print("Median: " + str(x))
x = getModus(main_df['Cholesterol_Level'])
print("Modus: " + str(x))
x = getStd(main_df['Cholesterol_Level'])
print("Standar Deviasi: " + str(x))
x = getVariance(main_df['Cholesterol_Level'])
print("Variansi: " + str(x))
x = getRange(main_df['Cholesterol_Level'])
print("Range: " + str(x))
x = getMin(main_df['Cholesterol_Level'])
print("Nilai Minimum: " + str(x))
x = getMax(main_df['Cholesterol_Level'])
print("Nilai Maksimum: " + str(x))
x = getKuartil(main_df['Cholesterol_Level'])
print("Kuartil: " + str(x))
x = getInterKuartil(main_df['Cholesterol_Level'])
print("Inter Kuartil: " + str(x))
x = getSkewness(main_df['Cholesterol_Level'])
print("Skewness: " + str(x))
x = kurtosis(main_df['Cholesterol_Level'])
print("Kurtosis: " + str(x))

Rata-rata: 4889928.319474324
Median: 4898900.675226251
Modus: 3093456.839129376
Standar Deviasi: 2010292.2629119065
Variansi: 4041274982323.4736
Range: 13316478.756404964
Nilai Minimum: -1088986.5264049638
Nilai Maksimum: 12227492.23
Kuartil: [3579592.598647509, 4898900.675226251, 6272831.370186756]
Inter Kuartil: 2693238.7715392476
Skewness: -0.006346172850509155
Kurtosis: -0.03448157628069426


### H) Deskripsi Statistika Weight

In [ ]:
main_df['Weight']

0       3.093457e+06
1       5.545445e+06
2       4.401808e+06
3       4.606865e+06
4       6.956049e+06
            ...     
1995    2.072546e+06
1996    7.456909e+06
1997    5.560391e+06
1998    3.436759e+06
1999    3.343102e+06
Name: Income, Length: 2000, dtype: float64

In [ ]:
x = getMean(main_df['Weight'])
print("Rata-rata: " + str(x))
x = getMedian(main_df['Weight'])
print("Median: " + str(x))
x = getModus(main_df['Weight'])
print("Modus: " + str(x))
x = getStd(main_df['Weight'])
print("Standar Deviasi: " + str(x))
x = getVariance(main_df['Weight'])
print("Variansi: " + str(x))
x = getRange(main_df['Weight'])
print("Range: " + str(x))
x = getMin(main_df['Weight'])
print("Nilai Minimum: " + str(x))
x = getMax(main_df['Weight'])
print("Nilai Maksimum: " + str(x))
x = getKuartil(main_df['Weight'])
print("Kuartil: " + str(x))
x = getInterKuartil(main_df['Weight'])
print("Inter Kuartil: " + str(x))
x = getSkewness(main_df['Weight'])
print("Skewness: " + str(x))
x = kurtosis(main_df['Weight'])
print("Kurtosis: " + str(x))

Rata-rata: 4889928.319474324
Median: 4898900.675226251
Modus: 3093456.839129376
Standar Deviasi: 2010292.2629119065
Variansi: 4041274982323.4736
Range: 13316478.756404964
Nilai Minimum: -1088986.5264049638
Nilai Maksimum: 12227492.23
Kuartil: [3579592.598647509, 4898900.675226251, 6272831.370186756]
Inter Kuartil: 2693238.7715392476
Skewness: -0.006346172850509155
Kurtosis: -0.03448157628069426


### I) Deskripsi Statistika Height

In [ ]:
main_df['Height']

0       3.093457e+06
1       5.545445e+06
2       4.401808e+06
3       4.606865e+06
4       6.956049e+06
            ...     
1995    2.072546e+06
1996    7.456909e+06
1997    5.560391e+06
1998    3.436759e+06
1999    3.343102e+06
Name: Income, Length: 2000, dtype: float64

In [ ]:
x = getMean(main_df['Height'])
print("Rata-rata: " + str(x))
x = getMedian(main_df['Height'])
print("Median: " + str(x))
x = getModus(main_df['Height'])
print("Modus: " + str(x))
x = getStd(main_df['Height'])
print("Standar Deviasi: " + str(x))
x = getVariance(main_df['Height'])
print("Variansi: " + str(x))
x = getRange(main_df['Height'])
print("Range: " + str(x))
x = getMin(main_df['Height'])
print("Nilai Minimum: " + str(x))
x = getMax(main_df['Height'])
print("Nilai Maksimum: " + str(x))
x = getKuartil(main_df['Height'])
print("Kuartil: " + str(x))
x = getInterKuartil(main_df['Height'])
print("Inter Kuartil: " + str(x))
x = getSkewness(main_df['Height'])
print("Skewness: " + str(x))
x = kurtosis(main_df['Height'])
print("Kurtosis: " + str(x))

Rata-rata: 4889928.319474324
Median: 4898900.675226251
Modus: 3093456.839129376
Standar Deviasi: 2010292.2629119065
Variansi: 4041274982323.4736
Range: 13316478.756404964
Nilai Minimum: -1088986.5264049638
Nilai Maksimum: 12227492.23
Kuartil: [3579592.598647509, 4898900.675226251, 6272831.370186756]
Inter Kuartil: 2693238.7715392476
Skewness: -0.006346172850509155
Kurtosis: -0.03448157628069426


### J) Deskripsi Statistika Blood Pressure

In [ ]:
main_df['Blood_Pressure']

0       3.093457e+06
1       5.545445e+06
2       4.401808e+06
3       4.606865e+06
4       6.956049e+06
            ...     
1995    2.072546e+06
1996    7.456909e+06
1997    5.560391e+06
1998    3.436759e+06
1999    3.343102e+06
Name: Income, Length: 2000, dtype: float64

In [ ]:
x = getMean(main_df['Blood_Pressure'])
print("Rata-rata: " + str(x))
x = getMedian(main_df['Blood_Pressure'])
print("Median: " + str(x))
x = getModus(main_df['Blood_Pressure'])
print("Modus: " + str(x))
x = getStd(main_df['Blood_Pressure'])
print("Standar Deviasi: " + str(x))
x = getVariance(main_df['Blood_Pressure'])
print("Variansi: " + str(x))
x = getRange(main_df['Blood_Pressure'])
print("Range: " + str(x))
x = getMin(main_df['Blood_Pressure'])
print("Nilai Minimum: " + str(x))
x = getMax(main_df['Blood_Pressure'])
print("Nilai Maksimum: " + str(x))
x = getKuartil(main_df['Blood_Pressure'])
print("Kuartil: " + str(x))
x = getInterKuartil(main_df['Blood_Pressure'])
print("Inter Kuartil: " + str(x))
x = getSkewness(main_df['Blood_Pressure'])
print("Skewness: " + str(x))
x = kurtosis(main_df['Blood_Pressure'])
print("Kurtosis: " + str(x))

Rata-rata: 4889928.319474324
Median: 4898900.675226251
Modus: 3093456.839129376
Standar Deviasi: 2010292.2629119065
Variansi: 4041274982323.4736
Range: 13316478.756404964
Nilai Minimum: -1088986.5264049638
Nilai Maksimum: 12227492.23
Kuartil: [3579592.598647509, 4898900.675226251, 6272831.370186756]
Inter Kuartil: 2693238.7715392476
Skewness: -0.006346172850509155
Kurtosis: -0.03448157628069426


### K) Deskripsi Statistika Health Status

In [ ]:
main_df['Health_Status']

0       Excellent
1            Fair
2       Excellent
3            Poor
4            Fair
          ...    
1995         Fair
1996    Excellent
1997         Poor
1998         Good
1999         Poor
Name: Health_Status, Length: 2000, dtype: object

In [ ]:
x = getUnique(main_df['Health_Status'])
if len(x)==0 :
  print("Tidak ada value unik")
else:
  print("Value Unik: " + str(x))

x = getProportion(main_df['Health_Status'])
print("Proporsi: " + str(x))

Value Unik: {'Excellent', 'Fair', 'Poor', 'Good'}
Proporsi: {'Excellent': 0.2515, 'Fair': 0.2595, 'Poor': 0.252, 'Good': 0.237}


## **2. Outlier**
Apakah pada data tersebut terdapat outlier? Jika ya, dapatkah anda menanganinya? Jelaskan apa yang umumnya dilakukan untuk menangani outlier.

## **3. Visualisasi Plot Distribusi**
Membuat Visualisasi plot distribusi. Berikan uraian penjelasan kondisi setiap kolom berdasarkan kedua plot tersebut. Jika numerik dapat dibuat dalam bentuk histogram dan box plot, dan jika string dengan histogram.

## **4. Menentukan Distribusi**
Menentukan distribusi setiap kolom numerik menggunakan hasil visualisasi histogram. Apakah kolom tersebut berdistribusi normal? Jika bukan, terdistribusi seperti apa kolom tersebut?
